# Spaceship Titanic

- Utilizaando os [dados disponíveis no Kaggle](https://www.kaggle.com/competitions/spaceship-titanic)
    - Dataset de **competição**
    - O resultado é avaliado através da **acurácia**

### Importando novamente as bases e fazendo o tratando dos dados
- Importando oque foi feito na [Parte 0 - Tratando os dados](https://github.com/PedroALage/Projetos/blob/main/Data_Science/Spaceship_Titanic/Parte0_TratandoDados.ipynb)

In [1]:
# Importando o pandas
import pandas as pd

In [2]:
# Visualizando a base de treino
treino = pd.read_csv('treino_trat1.csv')
treino.head(3)

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False


In [3]:
# Visualizando a base de teste
teste = pd.read_csv('teste_trat1.csv')
teste.head(3)

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,0013_01,Earth,True,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0
1,0018_01,Earth,False,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0
2,0019_01,Europa,True,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0


In [4]:
treino.dtypes

PassengerId      object
HomePlanet       object
CryoSleep          bool
Destination      object
Age             float64
VIP                bool
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Transported        bool
dtype: object

### Entendendo as colunas booleans
- Adicionando um novo tratamento das colunas do tipo bool
- Não será necessario tratar o Transported por se tratar do resultado

In [5]:
# Verificando as colunas de bool na base de treino
treino.columns[treino.dtypes == 'bool']

Index(['CryoSleep', 'VIP', 'Transported'], dtype='object')

In [6]:
# Verificando os valores na coluna CryoSleep
treino.CryoSleep.value_counts()

CryoSleep
False    5558
True     3135
Name: count, dtype: int64

In [7]:
# Verificando os valores na coluna VIP
treino.VIP.value_counts()

VIP
False    8494
True      199
Name: count, dtype: int64

- Para tratar as colunas, podemos utilizar astype(int) para criar uma nova coluna que **vai receber 1 se o campo for True e 0 se for False**

#### VIP

In [8]:
# Usando o astype(int) para fazer esse tratamento
treino['VIPCheck'] = treino['VIP'].astype(int)

In [9]:
# Verificando os valores
treino[['VIP', 'VIPCheck']].value_counts()

VIP    VIPCheck
False  0           8494
True   1            199
Name: count, dtype: int64

- Fazendo o mesmo para a base de teste

In [10]:
# Usando o astype(int) para fazer esse tratamento
teste['VIPCheck'] = teste['VIP'].astype(int)

In [11]:
# Verificando os valores
teste[['VIP', 'VIPCheck']].value_counts()

VIP    VIPCheck
False  0           4203
True   1             74
Name: count, dtype: int64

#### CryoSleep

In [12]:
# Usando o astype(int) para fazer esse tratamento
treino['CryoSCheck'] = treino['CryoSleep'].astype(int)

In [13]:
# Verificando os valores
treino[['CryoSleep', 'CryoSCheck']].value_counts()

CryoSleep  CryoSCheck
False      0             5558
True       1             3135
Name: count, dtype: int64

- Fazendo o mesmo para a base de teste

In [14]:
# Usando o astype(int) para fazer esse tratamento
teste['CryoSCheck'] = teste['CryoSleep'].astype(int)

In [15]:
# Verificando os valores
teste[['CryoSleep', 'CryoSCheck']].value_counts()

CryoSleep  CryoSCheck
False      0             2695
True       1             1582
Name: count, dtype: int64

### Entendendo as colunas de texto
- Adicionando um novo tratamento das colunas de texto

In [16]:
# Verificando as colunas de texto na base de treino
treino.columns[treino.dtypes == 'object']

Index(['PassengerId', 'HomePlanet', 'Destination'], dtype='object')

In [17]:
# Verificando os valores na coluna HomePlanet
treino.HomePlanet.value_counts()

HomePlanet
Earth     4803
Europa    2131
Mars      1759
Name: count, dtype: int64

In [18]:
# Verificando os valores na coluna Destination
treino.Destination.value_counts()

Destination
TRAPPIST-1e      6097
55 Cancri e      1800
PSO J318.5-22     796
Name: count, dtype: int64

- Para tratar as colunas será utilizado o **[OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)** que irá criar uma nova coluna para cada um dos rótulos da coluna original.

In [19]:
# Importando o OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

In [20]:
# Criando o encoder
ohe = OneHotEncoder(handle_unknown='ignore', dtype='int32')

#### HomePlanet

In [21]:
# Fazendo o fit com os dados
ohe_HP = ohe.fit(treino[['HomePlanet']])

In [22]:
# Fazendo a transformação
ohe_HP_array = ohe_HP.transform(treino[['HomePlanet']]).toarray()

In [23]:
# Transformando esse resultado em um DataFrame
ohe_HP_df = pd.DataFrame(ohe_HP_array, columns=ohe_HP.get_feature_names_out())
ohe_HP_df.head(3)

,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars
0,0,1,0
1,1,0,0
2,0,1,0


In [24]:
# Podemos agora adicionar essa coluna na nossa base de treino
treino = pd.concat([treino, ohe_HP_df], axis=1)

In [25]:
# Verificando os valores
treino[['HomePlanet','HomePlanet_Earth','HomePlanet_Europa','HomePlanet_Mars']].value_counts()

HomePlanet  HomePlanet_Earth  HomePlanet_Europa  HomePlanet_Mars
Earth       1                 0                  0                  4803
Europa      0                 1                  0                  2131
Mars        0                 0                  1                  1759
Name: count, dtype: int64

**HomePlanet** base de teste
- Não e necessário fazer o fit novamente

In [26]:
# Transformando esse resultado em um DataFrame
ohe_HP_array = ohe_HP.transform(teste[['HomePlanet']]).toarray()
ohe_HP_df = pd.DataFrame(ohe_HP_array, columns=ohe_HP.get_feature_names_out())

In [27]:
# Adicionando o resultado na base de teste
teste = pd.concat([teste, ohe_HP_df], axis=1)

In [28]:
# Verificando também os valores
teste[['HomePlanet','HomePlanet_Earth','HomePlanet_Europa','HomePlanet_Mars']].value_counts()

HomePlanet  HomePlanet_Earth  HomePlanet_Europa  HomePlanet_Mars
Earth       1                 0                  0                  2350
Europa      0                 1                  0                  1002
Mars        0                 0                  1                   925
Name: count, dtype: int64

#### Destination

In [29]:
# Fazendo o fit com os dados
ohe_DT = ohe.fit(treino[['Destination']])

In [30]:
# Fazendo a transformação
ohe_DT_array = ohe_DT.transform(treino[['Destination']]).toarray()

In [31]:
# Transformando esse resultado em um DataFrame
ohe_DT_df = pd.DataFrame(ohe_DT_array, columns=ohe_DT.get_feature_names_out())
ohe_DT_df.head(3)

,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0,0,1
1,0,0,1
2,0,0,1


In [32]:
# Podemos agora adicionar essa coluna na nossa base de treino
treino = pd.concat([treino, ohe_DT_df], axis=1)

In [33]:
# Verificando também os valores
treino[['Destination','Destination_55 Cancri e','Destination_PSO J318.5-22','Destination_TRAPPIST-1e']].value_counts()

Destination    Destination_55 Cancri e  Destination_PSO J318.5-22  Destination_TRAPPIST-1e
TRAPPIST-1e    0                        0                          1                          6097
55 Cancri e    1                        0                          0                          1800
PSO J318.5-22  0                        1                          0                           796
Name: count, dtype: int64

**Destination** base de teste
- Não é necessário fazer o fit novamente

In [34]:
# Transformando esse resultado em um DataFrame
ohe_DT_array = ohe_DT.transform(teste[['Destination']]).toarray()
ohe_DT_df = pd.DataFrame(ohe_DT_array, columns=ohe_DT.get_feature_names_out())

In [35]:
# Adicionando o resultado na base de teste
teste = pd.concat([teste, ohe_DT_df], axis=1)

In [36]:
# Verificando também os valores
teste[['Destination','Destination_55 Cancri e','Destination_PSO J318.5-22','Destination_TRAPPIST-1e']].value_counts()

Destination    Destination_55 Cancri e  Destination_PSO J318.5-22  Destination_TRAPPIST-1e
TRAPPIST-1e    0                        0                          1                          3048
55 Cancri e    1                        0                          0                           841
PSO J318.5-22  0                        1                          0                           388
Name: count, dtype: int64

### Usando a nova base no modelo

In [38]:
# Visualizando a base
treino.head(3)

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,VIPCheck,CryoSCheck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,0,0,0,1,0,0,0,1
1,0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,0,0,1,0,0,0,0,1
2,0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,1,0,0,1,0,0,0,1


In [39]:
# Podemos então apagar essas duas colunas que já tratamos
treino = treino.drop(['VIP', 'CryoSleep','HomePlanet', 'Destination'], axis=1)
teste = teste.drop(['VIP', 'CryoSleep','HomePlanet', 'Destination'], axis=1)

In [41]:
# Exportando para um csv
treino.to_csv('treino_trat2.csv', index=False)
teste.to_csv('teste_trat2.csv', index=False)

### Modelo para classificar os dados
- Testar entre:
    - **Árvore de classificação**
        - https://scikit-learn.org/stable/modules/tree.html#classification
    - **Classificação dos vizinhos mais próximos**
        - https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier
    - **Regressão Logística**
        - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression
- Antes de usar os algoritmos, precisamos separar a base de treino em **treino e validação**
    - Vamos fazer isso utilizando o **train_test_split**
        - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [42]:
# Importando o train_test_split
from sklearn.model_selection import train_test_split

In [43]:
# Separando a base de treino em X e y
X = treino.drop(['PassengerId','Transported'],axis=1)
y = treino.Transported

In [44]:
# Separando em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

- Para a **árvore de classificação**

In [45]:
# Fazendo a importação
from sklearn import tree

In [46]:
# Criando o classificador
clf_ac = tree.DecisionTreeClassifier()

In [47]:
# Fazendo o fit com os dados
clf_ac = clf_ac.fit(X_train,y_train)

In [48]:
# Fazendo a previsão
y_pred_ac = clf_ac.predict(X_val)

- Para o **KNeighborsClassifier**

In [49]:
# Importando
from sklearn.neighbors import KNeighborsClassifier

In [50]:
# Criando o classificador
clf_knn = KNeighborsClassifier(n_neighbors=3)

In [51]:
# Fazendo o fit com os dados
clf_knn = clf_knn.fit(X_train,y_train)

In [52]:
# Fazendo a previsão
y_pred_knn = clf_knn.predict(X_val)

- E para a **Regressão Logística**

In [53]:
# Importando
from sklearn.linear_model import LogisticRegression

In [54]:
# Criando o classificador
clf_rl = LogisticRegression(max_iter=1000)

In [55]:
# Fazendo o fit com os dados
clf_rl = clf_rl.fit(X_train,y_train)

In [56]:
# Fazendo a previsão
y_pred_rl = clf_rl.predict(X_val)

- Vamos novamente **avaliar os modelos**
    - Acurácia (método de avaliação usado na competição):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
    - Matriz de confusão (ajuda a visualizar a distribuição dos erros):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

- Avaliando a **acurácia**

In [57]:
# Importando
from sklearn.metrics import accuracy_score

In [58]:
# Para a árvore
accuracy_score(y_val, y_pred_ac)

0.7333565702335308

In [59]:
# Para o knn
accuracy_score(y_val, y_pred_knn)

0.7549668874172185

In [60]:
# Para a regressão logística
accuracy_score(y_val, y_pred_rl)

0.7765772046009063

- Avaliando a **matriz de confusão**

In [61]:
# Importando
from sklearn.metrics import confusion_matrix

In [62]:
# Para a árvore
confusion_matrix(y_val, y_pred_ac)

array([[ 977,  447],
       [ 318, 1127]], dtype=int64)

In [63]:
# Para o knn
confusion_matrix(y_val, y_pred_knn)

array([[1045,  379],
       [ 324, 1121]], dtype=int64)

In [64]:
# Para a regressão logística
confusion_matrix(y_val, y_pred_rl)

array([[1051,  373],
       [ 268, 1177]], dtype=int64)

### Fazendo a previsão para os dados de teste
- Vamos usar o modelo com melhor precisão para fazer o predict na base de teste

In [65]:
# Visualizando o X_train
X_train.head(3)

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,VIPCheck,CryoSCheck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
4696,35.0,1337.0,49.0,57.0,0.0,0.0,0,0,0,0,1,0,0,1
5946,28.0,0.0,152.0,215.0,30.0,510.0,0,0,1,0,0,0,0,1
227,43.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,1,0,0,1


In [66]:
# Visualizando a base de teste
teste.head(3)

,PassengerId,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,VIPCheck,CryoSCheck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0013_01,27.0,0.0,0.0,0.0,0.0,0.0,0,1,1,0,0,0,0,1
1,0018_01,19.0,0.0,9.0,0.0,2823.0,0.0,0,0,1,0,0,0,0,1
2,0019_01,31.0,0.0,0.0,0.0,0.0,0.0,0,1,0,1,0,1,0,0


In [67]:
# Para a base de teste ser igual a base de treino, precisamos eliminar a coluna de id
X_teste = teste.drop('PassengerId', axis=1)

In [68]:
# Utilizando a regressão logística na base de teste
y_pred = clf_rl.predict(X_teste)

In [69]:
# Criando uma nova coluna com a previsão na base de teste
teste['Transported'] = y_pred

In [70]:
# Selecionando apenas a coluna de Id e Survived para fazer o envio
base_envio = teste[['PassengerId', 'Transported']]

In [71]:
# Exportando para um csv
base_envio.to_csv('resultado2.csv', index=False)

## Resultado

- Modelo acertou 78,95% das avaliações
- Mesmo diminuindo a taxa de Falsos Negativos, o modelo não apresentou ganhos consideraveis em relação ao anterior
    
<img src="pkgImages/tentativa2.png" width=900>